# The Little Book of Semaphores
Working through my own implementations of concurrency and synchronization problems from the [Little Book of Semaphores](http://greenteapress.com/semaphores/LittleBookOfSemaphores.pdf) by A. B. Downey.

Will implement examples using asyncio. 

In [4]:
import asyncio
from IPython.core.debugger import set_trace

## Basic patterns

### Signaling
2 threads A and B, having to coordinate to do an action in a particular order.

In [45]:
async def action(name):
    return name
    
async def coroutineA(first_action_completed):
    """Do first action then signal to B that we're done"""
    asyncio.Task.current_task().name = "coroutineA"
    res = await action('A')
    first_action_completed.release()
    return res

async def coroutineB(first_action_completed):
    asyncio.Task.current_task().name = "coroutineB"
    async with first_action_completed:
        res = await action('B')
    return res
    
async def test_signaling(attempt):
    """Check that we're getting the """
    # Not using a Lock here because we want to signal/release before waiting/acquiring 
    # so we need a semaphore. 
    first_action_done = asyncio.Semaphore(0)
    res = await asyncio.gather(coroutineA(first_action_done), coroutineB(first_action_done))
    assert res == ['A', 'B'], f'Test failed for attempt {attempt}: got {res}'


async def run_test(test_coroutine, attempts):
    """Run a test multiple times to make sure we don't get lucky."""
    [await test_coroutine(attempt) for attempt in range(attempts)]

In [46]:
await run_test(test_signaling, 100)